In [ ]:
# Create Variables
$Password = "Passw0rd!"
$UserName = "testuser" # Can be also domain\testuser
$BasePath = "C:\GIT\Presentations\PowerShellUserGroup-2020-11-05"
$KeyPath = Join-Path -Path $BasePath -ChildPath "AES.key"
$PasswordPath = Join-Path -Path $BasePath -ChildPath "Password.txt"
$CLIPathIntegrated = Join-Path -Path $BasePath -ChildPath "CLIIntegrated.xml"
$CLIPathPSFramework = Join-Path -Path $BasePath -ChildPath "CLIPSFramework.xml"

$Password
$UserName
$BasePath
$KeyPath
$PasswordPath
$CLIPathIntegrated 
$CLIPathPSFramework

# PowerShell User Group innsalzach 05.11.2020

## Handling and managing Secrets using PowerShell

## Story:

John works at an MSP and has multiple customers. He stores his passwords in different places (KeyPass / Plain Text in Scripts / LastPass / Azure KeyVault...)

He wants to achieve the following goals:

-   Get rid of plain text passwords in scripts (Do not do that ever again. And yes API Keys and Azure Applications Secrets are also Passwords)
-   Access all of his Credential Vaults via PowerShell in an automated way (Problem: Master Password are in plain Text. What do we do about that?
-   Centralize the password management by just using one set of CmdLets and use only PowerShell (7/5.1)

## Basics of PowerShell Credentials

If you want to create a secret you need to things:  

- a UserName which is plain text (you can replace UserName with anything else)
- the password (needs to be a secure string)(the password could also be an API KEY or something else you want to store as secure string)

## Passwords in an interactive Shell

You can also enter the Password like this:

Problem: What do you do if this is used in a pipeline or scheduled task for example?

Only applicable for interactive scripts.

In [ ]:
# Interactive Password
$InteractivePassword = Read-Host -AsSecureString -Prompt "Please Enter Password"
$InteractivePassword

## If you want to use the PSCredential Object only on this computer use the following example:

### Create a secure string

In [ ]:
# Create a secure string
$SecPassword = ConvertTo-SecureString -String $Password -AsPlainText -Force

Let's look at the return value

In [ ]:
# Returnvalue
$SecPassword

Get The password again in clear Text

In [ ]:

ConvertFrom-SecureString -SecureString $SecPassword -AsPlainText

### Create a PSCredential Object using a securestring obejct

**Note**: The SecPassword Variable has to be always of type System.Security.SecureString. Please do not put a string an Argument to the PSCredential Object.

**Reference in the Microsoft Docs**: [PSCredential Object](https://docs.microsoft.com/en-us/dotnet/api/system.management.automation.pscredential?view=powershellsdk-7.0.0)

In [ ]:

$splat = @{
    TypeName = "System.Management.Automation.PSCredential"
    ArgumentList = $UserName, $SecPassword
}

$PSCred = New-Object @splat 
$PSCred

Create a PSCredential Object in an interactive way

In [ ]:

$Cred = Get-Credential -Message "Please enter your password"

If you want to reuse that credential you can export it to a file in an encrypted way

In [ ]:

# PowerShell integrated way
$Cred | Export-CliXML -Path $CLIPathIntegrated

# PSFramewor way
$Cred | Export-PSFClixml -Path $CLIPathPSFramework

Import the XML Files whenever neeeded

**Note:** This can only be done on the same machine

In [ ]:

# Import Credential using the integrated way
Import-Clixml -Path $CLIPathIntegrated

# Import Credential using PSFramework
Import-PSFClixml -Path $CLIPathPSFramework

## If you want to share a PSCredential with remote Computers also please use a key.  

**Note**: You can use that also if you want to use It only on one computer.

## Create a Key File

**Note**: If you want to make that key accessible to more computers store It somewhere safe on a fileshare. (Take care of the right ACLs)

In [ ]:

$KeyPath = Join-Path -Path $BasePath -ChildPath "AES.key"
$Key = New-Object -TypeName byte[] 16
[Security.Cryptography.RNGCryptoServiceProvider]::Create().GetBytes($Key)
$key | Out-File -FilePath $KeyPath

###  Create a SecureString and encrypt It with the key

In [ ]:

$key = Get-Content -Path $KeyPath
$EncPassword = ConvertTo-SecureString -String $Password -AsPlainText -Force
$EncPassword | ConvertFrom-SecureString -Key $key | Out-File -FilePath $PasswordPath

Reminder: Open Password File

### Create the PSCredential Object

In [ ]:

$key = Get-Content -Path $KeyPath
$pwd = Get-Content -Path $PasswordPath | ConvertTo-SecureString -Key $key
$PSCred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $UserName, $pwd

## Secrets Management and KeePass

KeePass ([KeePass Password Safe](https://keepass.info/)) is a widely uses Password Management System. It stores all Passwords into a .kdbx File.

In a .kdbx File you can have folders and Passwords. You can organize the passwords in this folders.

If you want to open the .kdbx File you have to provide a master password.

To access KeePass via PowerShell you have to install a module called [GitHub - PSKeePass/PoShKeePass: PowerShell module for KeePass](https://github.com/PSKeePass/PoShKeePass)

This module is only available in PowerShell 5.1 (Use PowerShell 5.1 for that)

In [ ]:
Install-Module -Name PoShKeePass

Let's take a look what Cmdlets do come with the module

In [ ]:

Get-Command -Module PoShKeePass

 So Let's create our KeePass Database and use a KeyFile for that

In [ ]:

New-KeePassDatabase -DatabasePath C:\PSUG\KeePass\PSCreateKeepass.kdbx -KeyPath C:\PSUG\KeePass\PSUGTest.key

Snap! That's not implemented yet. If you want to create that you have to do that through the GUI (Show that).

If you want to create a KeePass Database which is only protected by a master key you can create that with PowerShell

In [ ]:

$Credential = Get-Credential -UserName "NotNeeded" -Message "Please Provide the KeePass Master Key"
New-KeePassDatabase -DatabasePath C:\PSUG\KeePass\PSUGDatabaseWithMasterKeyOnly.kdbx -MasterKey $Credential

Now you can open the Database with PowerShell. To open the KeePass Database you first have to create a KeePass Database Configuration.

So Let's do that

In [ ]:

Get-KeePassDatabaseConfiguration | Remove-KeePassDatabaseConfiguration
New-KeePassDatabaseConfiguration -DatabaseProfileName PSUGTest -DatabasePath C:\PSUG\KeePass\Database.kdbx -KeyPath C:\PSUG\KeePass\PSUGTest.key
Get-KeePassDatabaseConfiguration

If you want to create a KeePass Database Configuration with a Master Key protected KeePass file

In [ ]:

Get-KeePassDatabaseConfiguration | Remove-KeePassDatabaseConfiguration
New-KeePassDatabaseConfiguration -DatabaseProfileName PSUGTest -DatabasePath C:\PSUG\KeePass\PSUGDatabaseWithMasterKeyOnly.kdbx -UseMasterKey
Get-KeePassDatabaseConfiguration

  

You can have multiple Database Configurations but if you want to set one as default

In [ ]:

Get-KeePassDatabaseConfiguration | Remove-KeePassDatabaseConfiguration
New-KeePassDatabaseConfiguration -DatabaseProfileName PSUGTest -DatabasePath C:\PSUG\KeePass\PSUGDatabaseWithMasterKeyOnly.kdbx -UseMasterKey -Default
Get-KeePassDatabaseConfiguration

If you want to set the default switch on a existing Database Configuration use (There is tab completion on the DatabaseProfileName)

In [ ]:

Update-KeePassDatabaseConfiguration -DatabaseProfileName PSUGTest -Default

Now we have to get the Root of our KeePass Database (With Master Key)

In [ ]:

$MasterKey = ConvertTo-SecureString -String ($Credential.GetNetworkCredential().Password) -AsPlainText -Force
$rootPath = (Get-KeePassGroup -MasterKey $MasterKey | Where-Object {-not $_.ParentGroup}).Name

Now we have to get the Root of our KeePass Database (With Key File)

In [ ]:

$rootPath = (Get-KeePassGroup | Where-Object {-not $_.ParentGroup}).Name

To Create a new Folder inside our KeePass Database (With Master Key)

In [ ]:

$MasterKey = ConvertTo-SecureString -String ($Credential.GetNetworkCredential().Password) -AsPlainText -Force
New-KeePassGroup -KeePassGroupParentPath $rootPath -KeePassGroupName "PSUG Folder" -MasterKey $MasterKey -ErrorAction SilentlyContinue

To Create a new Folder inside our KeePass Database (With Key File)

In [ ]:

New-KeePassGroup -KeePassGroupParentPath $rootPath -KeePassGroupName "PSUG Folder"

To Verify that we have created the folder inside our KeePass Database (With Master Key)

In [ ]:

$MasterKey = ConvertTo-SecureString -String ($Credential.GetNetworkCredential().Password) -AsPlainText -Force
Get-KeePassGroup -KeePassGroupPath "$rootPath/PSUG Folder" -MasterKey $MasterKey -ErrorAction SilentlyContinue

To Verify that we have created the folder inside our KeePass Database (With Key File)

In [ ]:

Get-KeePassGroup -KeePassGroupPath "$rootPath/PSUG Folder"

To Create a KeePass Password (With Master Key)

In [ ]:

$Passsword = Read-Host -AsSecureString -Prompt "Please enter the password"
New-KeePassEntry -KeePassEntryGroupPath $rootPath -Title "TestEntry" -UserName "TestUser" -KeePassPassword $Passsword -MasterKey $MasterKey -ErrorAction SilentlyContinue

To Create a KeePass Password (With Key File)

In [ ]:

$Passsword = Read-Host -AsSecureString -Prompt "Please enter the password"
New-KeePassEntry -KeePassEntryGroupPath $rootPath -Title "TestEntry" -UserName "TestUser" -KeePassPassword $Passsword

To Check if we realy created that entry (With Master Key)

In [ ]:

Get-KeePassEntry -KeePassEntryGroupPath $rootPath -Title TestEntry -MasterKey $MasterKey -ErrorAction SilentlyContinue

To Check if we realy created that entry (With Key File)

In [ ]:

Get-KeePassEntry -KeePassEntryGroupPath $rootPath -Title TestEntry

To get the entry back as PSCredential (With Master Key)

In [ ]:

$entry = Get-KeePassEntry -KeePassEntryGroupPath $rootPath -Title TestEntry -MasterKey $MasterKey -ErrorAction SilentlyContinue
$entry | Get-Member

$EntryToPSCred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $entry.UserName, $entry.Password
$EntryToPSCred | Get-Member

To get the entry back as PSCredential (With Key File)

In [ ]:

$entry = Get-KeePassEntry -KeePassEntryGroupPath $rootPath -Title TestEntry
$entry | Get-Member

$EntryToPSCred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $entry.UserName, $entry.Password
$EntryToPSCred | Get-Member

##  Secretsmanagement and Azure KeyVault

 To get access to our Azure Subscription we need to have the Az Modules installed

In [ ]:
# Install Az PowerShell Module
Install-Module -Name Az -Force

Now we need to Authenticate with Azure Credentials (Do that outside of Azure Data Studio in PowerShell 7 and login with chager...)

In [ ]:
# Authenticate to Azure
Connect-AzAccount

Now let us create an Azure KeyVault

In [ ]:
# Create an Azure Resource Group
$RG = New-AzResourceGroup -Name "PSUG-rg" -Location "West Europe"

# Create an Azure Key Vault
$KV = New-AzKeyVault -Name "PSUGVault" -ResourceGroupName $RG.ResourceGroupName -Location "West Europe"

Now that we created the key vault we have to assign permissions to a user to retrieve that secrets

In [ ]:
# Set the access policy
Set-AzKeyVaultAccessPolicy -VaultName $KV.VaultName -ObjectId "b6c01b99-648f-43a7-b3a9-ddc36acabefc" -PermissionsToSecrets get,set,delete,list -BypassObjectIdValidation

Now that we assigned the access policies we create a secret

In [ ]:
# Create the secret that we need to add to Azure Key Vault
$secret = Read-Host -AsSecureString -Prompt "Please add you secret"

# Add that secret to the Azure Key Vault
$AZKVSecret = Set-AzKeyVaultSecret -VaultName $KV.VaultName -Name "PSUGSecret" -SecretValue $secret

To retrieve a secretvalue where you have access

In [ ]:
# Retrieve a SecretValue
(Get-AzKeyVaultSecret -VaultName $KV.VaultName -Name "PSUGSecret").SecretValueText

##  Secretmanagement mit Microsoft.PowerShell.SecretManagement

Source: [SecretManagement](https://devblogs.microsoft.com/powershell/secretmanagement-preview-3/)

The SecretManagement module helps users manage secrets by providing a common set of cmdlets to interface with secrets across vaults. 

This module supports an extensible model where local and remote vaults can be registered and unregistered for use in accessing and retrieving secrets.

SecretManagement is valuable in heterogeneous environments where you may want to separate the specifics of the vault from a common script which needs secrets. 

SecretManagement is also as a convenience feature which allows users to simplify their interactions with various vaults by only needing to learn a single set of cmdlets.

Some key scenarios we have heard from PowerShell users are:

-   Sharing a script across my org (or Open Source) without knowing the platform/local vault of all the users
-   Running my deployment script in local, test and production with the change of only a single parameter (-Vault)
-   Changing the backend of the authentication method to meet specific security or organizational needs without needing to update all my scripts

Install The Microsoft.PowerShell.SecretManagement Module

In [ ]:
# Install the SecretManagement Module
Install-Module -Name Microsoft.PowerShell.SecretManagement -AllowPrerelease

So Let us get all the command of that Module

In [ ]:
# Retrieve all the CmdLets
Get-Command -Module Microsoft.PowerShell.SecretManagement

So. We can start? No we cannot because there is no vault registered.

In [4]:
# Trying to retrieve Vaults
Get-SecretVault

So Check what is in the PSGallery (Look for a Tag SecretManagement. Tip: If you want to create your own SecretsManagement Vault Tag It with SecretManagement)

In [5]:
# Find all the Vaults in the gallery.
Find-Module -Tag SecretManagement

Version              Name                                Repository           Description


-------              ----                                ----------           -----------


0.1.0                SecretManagement.LastPass           PSGallery            SecretManagement ext…


0.0.4.4              SecretManagement.KeePass            PSGallery            A cross-platform Kee…


0.1.1                SecretManagement.KeyChain           PSGallery            SecretManagement ext…


There is one more thing... Microsoft implemented It's own SecretManagement Vault. You Can Find It here

In [ ]:
# Find the SecretStore Module
Find-Module Microsoft.PowerShell.SecretStore -AllowPrerelease

 Fairly New: There is a Hashicorp Vault SecretManagement Store

In [ ]:
# Find Hashicorp Vault SecretVault
Find-Module -Name SecretManagement.Hashicorp.Vault.KV -AllowPrerelease

So Let's install the SecretStore, LastPass, AxKeyVault and the KeePass Vault

In [ ]:
# Install the 3 SecretManagementVault
Install-Module -Name SecretManagement.LastPass, SecretManagement.KeePass
Install-Module -Name Microsoft.PowerShell.SecretStore -AllowPrerelease

So Let's Register all of the vaults

In [ ]:
Register-SecretVault -Name SecretStore -ModuleName Microsoft.PowerShell.SecretStore
Register-SecretVault -Name LastPass -ModuleName SecretManagement.LastPass -VaultParameters @{
    lpassCommand = 'wsl'
}
Register-SecretVault -Name KeePass -ModuleName SecretManagement.KeePass -VaultParameters @{
    Path = 'C:\PSUG\KeePass\PSUGDatabaseWithMasterKeyOnly.kdbx'
}
# Currently Not working
Register-SecretVault -Name AzKeyVault -ModuleName Az.KeyVault -VaultParameters @{
    AZKVaultName = 'chwvdsecrets'
    SubscriptionId = '3932848d-e8f8-4a6e-a772-bc69214d0530'
}

### LastPass Secretmanagement

So To use the LastPass extension you need to have the lpass-cli installed. Lpass-cli does not run on Windows.

I installed WSL2 and installed lpass-cli into it. That's why I have to supply a vault parameter. After that

you have to run **wsl lpass login -trust** . You have to supply your master password and two factor If

enabled. After that you can run something like this:

In [ ]:
# Getting a Secret
$cred = Get-Secret -Name PSUG -Vault LastPass

# Looking at the Type
$cred | Get-Member

# Getting the Password
$cred.GetNetworkCredential().Password

TODO: Secret Management,LastPass

## Housekeeping

In [ ]:

Remove-Item -Path $KeyPath -Force
Remove-Item -Path $PasswordPath -Force
Remove-Item -Path $CLIPathIntegrated -Force
Remove-Item -Path $CLIPathPSFramework -Force

Get-KeePassDatabaseConfiguration | Remove-KeePassDatabaseConfiguration
Remove-Item -Path C:\PSUG -Recurse -Force

Remove-AzResourceGroup -Name PSUG-rg -Force